In [87]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [88]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [89]:
learning_rate=0.001
training_epochs=30
batch_size=100

In [90]:
cifar10_train = dsets.CIFAR10(root='data/cifar10_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
cifar10_test = dsets.CIFAR10(root='data/cifar10_data/',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

Files already downloaded and verified
Files already downloaded and verified


In [91]:
data_loader = DataLoader(dataset=cifar10_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [92]:
data_test = DataLoader(dataset=cifar10_test,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [93]:
class CNN(nn.Module):
  def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # self.layer3 = nn.Sequential(
        #     nn.Conv2d(16, 128, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(8*8*16, 64, bias=True)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        nn.init.uniform_(self.fc1.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    # out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out


In [94]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [95]:
total_batch = len(data_loader)
print(total_batch)

500


In [96]:
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    y_hat=model(X)
    cost=criterion(y_hat, Y)

    cost.backward()
    optimizer.step()
    avg_cost+=cost/total_batch
  print(epoch, avg_cost)

0 tensor(2.3524, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(1.7536, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(1.5886, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(1.5096, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(1.4467, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(1.3998, device='cuda:0', grad_fn=<AddBackward0>)
6 tensor(1.3561, device='cuda:0', grad_fn=<AddBackward0>)
7 tensor(1.3202, device='cuda:0', grad_fn=<AddBackward0>)
8 tensor(1.2772, device='cuda:0', grad_fn=<AddBackward0>)
9 tensor(1.2488, device='cuda:0', grad_fn=<AddBackward0>)
10 tensor(1.2182, device='cuda:0', grad_fn=<AddBackward0>)
11 tensor(1.1945, device='cuda:0', grad_fn=<AddBackward0>)
12 tensor(1.1726, device='cuda:0', grad_fn=<AddBackward0>)
13 tensor(1.1555, device='cuda:0', grad_fn=<AddBackward0>)
14 tensor(1.1436, device='cuda:0', grad_fn=<AddBackward0>)
15 tensor(1.1244, device='cuda:0', grad_fn=<AddBackward0>)
16 tensor(1.1170, device='cuda:0', grad_fn=<AddBackward0>)
17 tens

In [105]:
with torch.no_grad():
  X_test = torch.from_numpy(cifar10_test.data.reshape(len(cifar10_test), 3, 32, 32)).float().to(device)
  Y_test = torch.LongTensor(cifar10_test.targets).to(device)
  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuray: ', accuracy.item())

Accuray:  0.08509999513626099


In [106]:
print(cifar10_train.targets[:50])
print(cifar10_test.targets[:50])
sum = 0
for i in range(500):
  if cifar10_train.targets[i] == cifar10_test.targets[i]:
    sum += 1

print(sum)
print(46/500)

[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0]
[3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6]
46
0.092
